In [1]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from sklearn.datasets import make_circles
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
from tqdm.notebook import tqdm
import copy

# Generate Data with Gaussian Process signal

In [2]:
import timesynth as ts

In [3]:
def seasonal(t, amplitude, period):
    """Generate a sinusoidal curve."""
    return amplitude * np.sin((2*np.pi*t)/period)

In [4]:
n = 146 
t0 = np.arange(146)

epsilon = np.random.normal(loc=0, scale=1.0, size=146)
s2 = seasonal(t0, 0.8, 30)
s3 = np.exp(-((t0 - 40) / 10)**2) * 10

Bz = s3 + s2 + epsilon

plt.figure(figsize=(18, 5))
plt.plot(t0, Bz)

In [5]:
s4 = -np.exp(-((t0 - 40) / 10)**2) * 10
DST = s4 + s2 + epsilon
plt.figure(figsize=(18, 5))
plt.plot(t0, DST)

# Data Augmentation & Preparation
Data normalization between [-1, 1]
$$
x^{''} = 2\frac{x - \text{min}(x)}{\text{max}(x) - \text{min}(x)} - 1
$$

In [97]:
def normalize(x):
    return 2*((x - np.min(x)) / (np.max(x) - np.min(x))) - 1

y_train = normalize(DST)
x_train = normalize(Bz)

In [99]:
plt.plot(t0, x_train)
plt.plot(t0, y_train)

### 8h rolling window batch generation

In [100]:
def generate_batches(x, y):
    '''
    Function creates batches out of x data of size 8 hours,
    8-hour window is moved throughout the array with step one hour,
    and cuts first 8 hours out of y data, so that for every 8 hours of x data
    there is one hour of y data hour ahead of input neurons data. 
    
    Parameters
    ----------
    x - x training data (Bz, sigma_Bz, n, v...)
    y - y training data (DST)

    Returns
    -------
    batches - Matrix where rows correspond to 8 hours of x array data
          y - y training data (DST) that has first 8 hours cut off
    '''
    
    y = y[8:]
    y = torch.from_numpy(np.array(y))
    
    batches = []
    
    for i in range(146):
        if (i+8) <= 146:
            # print(i, i+8)
            batch = x[i:i+8]
            batches.append(batch)
    
    batches = batches[:-1]
    batches = torch.from_numpy(np.array(batches))
    
    return batches, y


In [101]:
x_train, y_train = generate_batches(x_train, y_train)

In [102]:
len(x_train), len(y_train), x_train.shape, y_train.shape

# FFNN

In [86]:
model = nn.Sequential(
    nn.Linear(8, 26),
    nn.Tanh(),
    nn.Linear(26,1),
)

In [87]:
alpha = 0.03
n_epochs = 40

loss_fn = nn.MSELoss()
optim = torch.optim.Adam(model.parameters(), lr=alpha)

best_mse = np.inf
best_weights = None

history = []

In [103]:
curr_loss = None
for i in range(n_epochs):
    model.train()
    for batch, y in zip(x_train, y_train):
        batch = batch.to(torch.float32)
        y = y.to(torch.float32)

        y_pred  = model(batch)
        loss    = loss_fn(y_pred, y)
        curr_loss = loss
        
        optim.zero_grad()
        loss.backward()

        optim.step()
        
    print(curr_loss)
    history.append(curr_loss)

In [104]:
model.eval()

In [105]:
preds = []
for batch in x_train:
    batch = batch.float()
    preds.append(model(batch).detach().numpy())
preds = np.array(preds)
len(preds)

In [106]:
plt.plot(t0[8:], preds, label='prediction')
plt.plot(t0[8:], y_train, label='real')
plt.legend()